In [1]:
import sys
sys.path.insert(0,'../')
import rg
from rg.diagrams import diagram, composition_diagram, diagram_set
from rg.interaction import interaction as J 
from rg.interaction import interaction_identity
from rg.interaction import interaction_system
from rg.interaction import composite_interaction as G
from rg.theory import ftheory
from rg.graphs import composite_interaction_graph as fgraph
from rg.propagator import cpropagator, kpropagator

import numpy as np
from sympy import init_printing
init_printing()

# Create some sample primitives

In [2]:
coagulation = J([[2,1],[0,0]])
coagulation.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [3]:
branching = J([[1,2],[0,0]])
branching.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(0.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [4]:
cross = J([[1,1],[1,1]])
cross.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-210.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-150.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

In [5]:
branching_alt = J([[1,1],[0,1]])
coagulation_alt = J([[1,1],[1,0]])
coagulation_alt.diagram

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="240" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round">
         <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-30.0 0, 0 )" />  <path d="M 0,0 C 2,-3.142 3,-5 5,-5 S 8,-3.142 10,0 S 13,5 15,5 S 18,3.142 20,0 C 22,-3.142 23,-5 25,-5 S 28,-3.142 30,0 S 33,5 35,5 S 38,3.142 40,0" 
                       stroke="black" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(30.0 0, 0 )" />  <line x1="0" y1="0" x2="40" y2="0" stroke="red" stroke-dasharray="None" stroke-width:2" transform=" translate(80,50) rotate(-180.0 0, 0 )" /> 
      </g>
      <circle cx="80" cy="50" r="2" stroke="black" stroke-width="1" fill="black" />
    </svg>

# Work with two example graphs

In [6]:
U = G(branching)*G(cross)*G(coagulation)
composition_diagram(U,compact=True)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="120" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="70" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="102" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="102" cy="59" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="138" cy="50" r="2" stroke="green" stroke-width="1"  /> <circle cx="88" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="52" cy="41" r="2" stroke="green" stroke-width="1" fill='green' /> <circle cx="52" cy="59" r="2" stroke="orange" stroke-width="1" fill='orange' /> <circle cx="88" cy="59" r="2" stroke="orange" stroke-width="1"  /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill='green' /> <line x1="102" y1="41" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="41" x2="88" y2="41" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="88" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="120" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="102" y1="59" x2="38" y2="41" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="41" x2="70" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="52" y1="41" x2="38" y2="59" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"  stroke-dasharray='2,2' /><line x1="138" y1="50" x2="120" y2="50" stroke="green" stroke-width:2"   /><line x1="52" y1="59" x2="70" y2="50" stroke="orange" stroke-width:2"   /><line x1="88" y1="59" x2="70" y2="50" stroke="orange" stroke-width:2"   /><line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>

In [7]:
#composition_diagram(T,compact=True)

# The graph theory stuff is in fgraph - e.g. Incidence Matrix

In [8]:
GU = fgraph(U,True)
GU

$\left[\begin{smallmatrix}-1 & -1 & 0 & -1 & 0 & 0 & 0\\1 & 0 & 1 & 0 & -1 & -1 & 0\\0 & 1 & -1 & 0 & 0 & 0 & -1\\0 & 0 & 0 & 1 & 1 & 1 & 1\end{smallmatrix}\right]$

In [9]:
GU.draw_decomp()

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="170">['<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="0" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="50" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="38" cy="59" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="40" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="41" r="2" stroke="green" stroke-width="1"  /> <circle cx="2" cy="59" r="2" stroke="orange" stroke-width="1" fill=\'orange\' /> <circle cx="38" cy="59" r="2" stroke="orange" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   /><line x1="38" y1="59" x2="20" y2="50" stroke="orange" stroke-width:2"   />  </g> </svg>', '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0" y="80" width="500" height="120">\n      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> <circle cx="20" cy="50" r="3" stroke="black" stroke-width="1" fill="black" /> <circle cx="2" cy="41" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="2" cy="59" r="2" stroke="green" stroke-width="1" fill=\'green\' /> <circle cx="38" cy="50" r="2" stroke="green" stroke-width="1"  /> <line x1="2" y1="41" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="2" y1="59" x2="20" y2="50" stroke="green" stroke-width:2"   /><line x1="38" y1="50" x2="20" y2="50" stroke="green" stroke-width:2"   />  </g> </svg>']</svg>

In [10]:
#GU = fgraph(T,True)#.betti_number()
GU.gamma_integral

                                            d                    
                                            ─ - 2                
           -d                               2                    
           ─── ⎛     2                     ⎞                     
            2  ⎜D₀⋅kₚ  - 2⋅ⅈ⋅\omegaₚ + 4⋅m₀⎟            ⎛  d    ⎞
1.0⋅(4⋅\pi)   ⋅⎜───────────────────────────⎟     ⋅\Gamma⎜- ─ + 2⎟
               ⎝            4⋅D₀           ⎠            ⎝  2    ⎠
─────────────────────────────────────────────────────────────────
                            \Gamma(2)                            

### Convention:

While we could enforce more conventions earlier, here i assume a random edge ordering and now i want to enforce an ordering

I choose to have all external momenta pointing out of the graph (where we connected to the infinity vertext) and start by assuming all flow is towards increasing vid (order of vertices added in construction)

Then i choose the shortest path from the sink vertex (last vertex added) to the source vertex (first added) and i reverse the orientation on the path

We already at this point have some convention- the first row of the inc matrix is the source, the second last is the sink and the last is the infinity vertex used to complete the graph

This would be done for all loops in a general graph but for now we assume one loop

In [11]:
#choose all edges on the shortest path between source and sink as the "backflow" edges. one of these is the loop momentum
g = fgraph.edges_to_adjaceny_dict(U.edges)
walk = fgraph.shortest_path(g, 0,2)
walk = list(fgraph.expand_path(walk))
g,walk

({0: [1, 2], 2: [1]}, [[0, 2]])

In [12]:
#SVG draw arrows on the compact rep, todo

## observe different sign states on the incident matrix 
we start off seeking the standard form, externals point out, internals point "forward" - by default this is not how it looks so we change edge direction in the 2nd matrix

then we invert the shortest path i.e. change the direction of any edge on the shortest path. now we have our standard form that we will generally assume

in this case, in final matrix, column 1 (0-indexed) is the only one that goes from 1 to -1 reading from top to bottom - this is the "backflow edge" in the loop

In [13]:
inc = GU._inc 
inc

array([[-1, -1,  0, -1,  0,  0,  0],
       [ 1,  0,  1,  0, -1, -1,  0],
       [ 0,  1, -1,  0,  0,  0, -1],
       [ 0,  0,  0,  1,  1,  1,  1]])

In [14]:
inc = fgraph.should_toggle_edge_columns(inc) * inc
inc

array([[-1, -1,  0, -1,  0,  0,  0],
       [ 1,  0, -1,  0, -1, -1,  0],
       [ 0,  1,  1,  0,  0,  0, -1],
       [ 0,  0,  0,  1,  1,  1,  1]])

In [15]:
inc_inv = fgraph.invert_path(inc, walk)
inc_inv

array([[-1,  1,  0, -1,  0,  0,  0],
       [ 1,  0, -1,  0, -1, -1,  0],
       [ 0, -1,  1,  0,  0,  0, -1],
       [ 0,  0,  0,  1,  1,  1,  1]])

## Evaluate the momentum equations

this convention above makes the interpretation of the momentum equations less ambiguous. we now keep signs for the momentum as per the incidence matrix.

In [16]:
GU.edge_system

([-p1 - q1 - q2, k - p2 - p3 + q1, -k - p4 + q2, p1 + p2 + p3 + p4],
 {p2: -p3 - p4 + q1 + q2, k: -p4 + q2, p1: -q1 - q2},
 {0: q1, 1: q2, 2: k, 3: p1, 4: p2, 5: p3, 6: p4})

In [17]:
#actually from the notes I am not sure about the following:
#1 if the integrals are the same for any external leg structure, then momenta are based only on the number of vertices/propagators - however how do we resolve momenta
#2 there are some trasmutation "vertices" in diags which supposedly have no local residuals and the loops have the same q structure as the 2-nodes - why is that
#3 there are integrals with 3+ vertices that have no dependance on external momenta but that does not add up for me
#fow now im going to do a hack because i cannot justify what we do in the notes; my rule is going to be for the 2-vertex loop I have one rule and for the 3 and above I have another rule
#it seems without some argument, the same way we have casuality differences which must have momenta differences on certain edges

## This leads to integration of graph momenta (consider simple case)...

In [18]:
# As i have not decided how k will be determined for sure, I leave it as a parameter here
propagators = cpropagator.from_edges(U.edges,k_node_index=1)
[p.display() for p in propagators]

⎡           1                                 1                               
⎢───────────────────────, ──────────────────────────────────────────, ────────
⎢     2                               2                                    2  
⎣D₀⋅kₗ  + ⅈ⋅\omegaₗ + m₀  D₀⋅(kₗ + kₚ)  + m₀ - ⅈ⋅(\omegaₗ + \omegaₚ)  D₀⋅kₗ  +

   1           ⎤
───────────────⎥
               ⎥
 ⅈ⋅\omegaₗ + m₀⎦

In [19]:
#this is just the first residue - in this case there is only one anyway
cpropagator.residues(propagators)[0]["value"]

                          -ⅈ                           
───────────────────────────────────────────────────────
                                                      2
⎛     2          ⎛                2                 ⎞⎞ 
⎝D₀⋅kₗ  + m₀ + ⅈ⋅⎝- ⅈ⋅D₀⋅(kₗ + kₚ)  - \omegaₚ - ⅈ⋅m₀⎠⎠ 

In [64]:
K = cpropagator.integrate(propagators)
K

                     1                     
───────────────────────────────────────────
  ⎛     2     ⎞ ⎛     2        2          ⎞
2⋅⎝D₀⋅kₗ  + m₀⎠⋅⎝D₀⋅kₗ  + D₁⋅kₗ  + m₀ + m₁⎠

In [65]:
PI = kpropagator(K)
PI.parametric_integral

                          1                           
──────────────────────────────────────────────────────
                                                     2
  ⎛   ⎛     2     ⎞      ⎛     2        2          ⎞⎞ 
2⋅⎝α₁⋅⎝D₀⋅kₗ  + m₀⎠ + α₂⋅⎝D₀⋅kₗ  + D₁⋅kₗ  + m₀ + m₁⎠⎠ 

In [71]:
PI.reduced_parametric_integral

                                 2                
                (D₀ - D₁⋅α₁ + D₁)                 
──────────────────────────────────────────────────
                                                 2
  ⎛     2           2        2                  ⎞ 
2⋅⎝D₀⋅kₗ  - D₁⋅α₁⋅kₗ  + D₁⋅kₗ  - α₁⋅m₁ + m₀ + m₁⎠ 

In [72]:
PI._p

Poly(k_l**2 + (-alpha_1*m_1 + m_0 + m_1)/(D_0 - D_1*alpha_1 + D_1), k_l, domai
n='ZZ(D_0,D_1,alpha_1,m_0,m_1)')

In [73]:
PI.used_chung_wu

False

In [76]:
terms = kpropagator.__reduced_terms__(PI.reduced_parametric_integral)
terms["M"] ,terms["r"], terms["nu"] , terms["prefactor"]

⎛-α₁⋅m₁ + m₀ + m₁         ⎞
⎜────────────────, 0, 2, 2⎟
⎝D₀ - D₁⋅α₁ + D₁          ⎠

In [77]:
PI.gamma_integral()

                             d                    
       -d                    ─ - 2                
       ───                   2                    
        2  ⎛-α₁⋅m₁ + m₀ + m₁⎞            ⎛  d    ⎞
(4⋅\pi)   ⋅⎜────────────────⎟     ⋅\Gamma⎜- ─ + 2⎟
           ⎝D₀ - D₁⋅α₁ + D₁ ⎠            ⎝  2    ⎠
──────────────────────────────────────────────────
                   2⋅\Gamma(2)                    

## Example with alpha params not needed for linear reduction => integrate on simplex

In [78]:
propagators = [cpropagator(0,1, -1),cpropagator(1,1,1),cpropagator(0,1, 1)]
[c.display() for c in propagators]

⎡           1                        1                        1           ⎤
⎢───────────────────────, ───────────────────────, ───────────────────────⎥
⎢     2                        2                        2                 ⎥
⎣D₀⋅kₗ  - ⅈ⋅\omegaₗ + m₀  D₁⋅kₗ  + ⅈ⋅\omegaₗ + m₁  D₀⋅kₗ  + ⅈ⋅\omegaₗ + m₀⎦

In [79]:
C = cpropagator.integrate(propagators)
C

                     1                     
───────────────────────────────────────────
  ⎛     2     ⎞ ⎛     2        2          ⎞
2⋅⎝D₀⋅kₗ  + m₀⎠⋅⎝D₀⋅kₗ  + D₁⋅kₗ  + m₀ + m₁⎠

In [80]:
K = kpropagator(C)
K.parametric_integral

                          1                           
──────────────────────────────────────────────────────
                                                     2
  ⎛   ⎛     2     ⎞      ⎛     2        2          ⎞⎞ 
2⋅⎝α₁⋅⎝D₀⋅kₗ  + m₀⎠ + α₂⋅⎝D₀⋅kₗ  + D₁⋅kₗ  + m₀ + m₁⎠⎠ 

In [81]:
K.reduced_parametric_integral

                                 2                
                (D₀ - D₁⋅α₁ + D₁)                 
──────────────────────────────────────────────────
                                                 2
  ⎛     2           2        2                  ⎞ 
2⋅⎝D₀⋅kₗ  - D₁⋅α₁⋅kₗ  + D₁⋅kₗ  - α₁⋅m₁ + m₀ + m₁⎠ 

In [82]:
K._p

Poly(k_l**2 + (-alpha_1*m_1 + m_0 + m_1)/(D_0 - D_1*alpha_1 + D_1), k_l, domai
n='ZZ(D_0,D_1,alpha_1,m_0,m_1)')

In [83]:
K.reduced_terms
#add constant prefactor and D prefactor for book keeping only

{'M': (-alpha_1*m_1 + m_0 + m_1)/(D_0 - D_1*alpha_1 + D_1),
 'nu': 2,
 'prefactor': 2,
 'r': 0}

In [84]:
K.gamma_integral()

                             d                    
       -d                    ─ - 2                
       ───                   2                    
        2  ⎛-α₁⋅m₁ + m₀ + m₁⎞            ⎛  d    ⎞
(4⋅\pi)   ⋅⎜────────────────⎟     ⋅\Gamma⎜- ─ + 2⎟
           ⎝D₀ - D₁⋅α₁ + D₁ ⎠            ⎝  2    ⎠
──────────────────────────────────────────────────
                   2⋅\Gamma(2)                    

In [85]:
K.used_chung_wu

False

In [86]:
K.gamma_integral(True, elim=["D_1"])
#decompose reduced terms into qm_kernel, qm_constants, d_constants,
#to isolate k^2 we have already factored out D^nu, now this will cancel with D^alpha=D^{d/2-nu} and we will have D^{d/2} and constants^{alpha}

                          d                    
  -d         -d           ─ - 1                
  ───        ───          2                    
   2          2  ⎛     m₁⎞            ⎛  d    ⎞
D₀   ⋅(4⋅\pi)   ⋅⎜m₀ + ──⎟     ⋅\Gamma⎜- ─ + 2⎟
                 ⎝     2 ⎠            ⎝  2    ⎠
───────────────────────────────────────────────
                ⎛d    ⎞                        
              2⋅⎜─ - 1⎟⋅\Gamma(2)              
                ⎝2    ⎠                        

In [87]:
#from sympy import *
#General form?
#integrate over simplex
#Symbol("\hat{D}")**(Symbol("d")/2) *Symbol("M")**(Symbol("D")/2-Symbol("nu"))

In [37]:
#from this factor out constants and construct a regular polynomial in the alpha parameters
#I have already reduced D by the power 

behind the scenes we just integrate the kernel directly rather than allowing sympy to integrate quasi mass

In [101]:
from sympy import integrate,Symbol,simplify
li = K.reduced_terms["M"]
li
#what if D1 is in the integral - how do we integrate that sensibly?

-α₁⋅m₁ + m₀ + m₁
────────────────
D₀ - D₁⋅α₁ + D₁ 

In [102]:
li.subs(Symbol("D_1"),0)

-α₁⋅m₁ + m₀ + m₁
────────────────
       D₀       

In [104]:
#get the alpha parameter - there is one of them in this case - and integrate over 0,1
a = [ a for a in li.atoms() if not a.is_constant() and a.name[:5]=="alpha"][0]
simplify(integrate(li.subs(Symbol("D_1"),0), (a,0,1)))
#after integrating the kernel we take into account the power and prefactors too

     m₁
m₀ + ──
     2 
───────
   D₀  

# Example with alpha parameters used for linear reduction

In [41]:
#punch in the example from theor. ref.

In [42]:
#what if we have something like this - actually before doing this, see if we can avoid this stage because it is essentially taking us
#out of a group where we had a standard propagator basis 
from sympy import symbols,I
k, p, m1, m2, w, D = symbols("k_l, k_p, m_1, m_2, \omega_p, D_0")
P1 =  (-k+p)**2 + (m1 + m2 -I*w )/D
P2 =  k**2+ (-k+p)**2 + (2*m1-I*w)/D
awkprops = list((1/P1,1/P2))
P = awkprops[0] * awkprops[1]
awkprops

⎡                1                                     1                  ⎤
⎢──────────────────────────────────, ─────────────────────────────────────⎥
⎢          2   -ⅈ⋅\omegaₚ + m₁ + m₂    2             2   -ⅈ⋅\omegaₚ + 2⋅m₁⎥
⎢(-kₗ + kₚ)  + ────────────────────  kₗ  + (-kₗ + kₚ)  + ─────────────────⎥
⎣                       D₀                                       D₀       ⎦

In [43]:
# propagators = [cpropagator(0,1,-1, ["l", "p"]), cpropagator(0,1,1, ["l", "p"], extra_mom_term=Symbol("k")**2) ]
# [p.display() for p in propagators]
# K = cpropagator.integrate(propagators)
# K

In [44]:
p2 = kpropagator(P)
p2.parametric_integral
#p2.reduced_parametric_integral

#TODO - alpha is gone, bring it back, and determine how to do the integration in each case - ALSO make sure we have a nice normalisation with nothing in numerator
#late rcompare with graph polynomials

                                               2                              
                                             D₀                               
──────────────────────────────────────────────────────────────────────────────
                                                                              
⎛   ⎛   ⎛  2             2⎞                   ⎞      ⎛             2          
⎝α₁⋅⎝D₀⋅⎝kₗ  + (-kₗ + kₚ) ⎠ - ⅈ⋅\omegaₚ + 2⋅m₁⎠ + α₂⋅⎝D₀⋅(-kₗ + kₚ)  - ⅈ⋅\omeg

               
               
───────────────
              2
            ⎞⎞ 
aₚ + m₁ + m₂⎠⎠ 

In [45]:
term = list(p2.parametric_integral.as_numer_denom()[1].as_powers_dict().keys())[0]

In [46]:
from sympy import * 
from rg.propagator import determine_elimination_variable
P = Poly(expand(term), Symbol("k_l"))
P

Poly((2*D_0*alpha_1 + D_0*alpha_2)*k_l**2 + (-2*D_0*alpha_1*k_p - 2*D_0*alpha_
2*k_p)*k_l + D_0*alpha_1*k_p**2 + D_0*alpha_2*k_p**2 - I*\omega_p*alpha_1 - I*
\omega_p*alpha_2 + 2*alpha_1*m_1 + alpha_2*m_1 + alpha_2*m_2, k_l, domain='EX'
)

In [47]:
#simpler = P.subs(determine_elimination_variable(P), 1)
simpler = P.subs(Symbol("alpha_1"), Symbol("alpha_2")-1)
simpler

        2        2                                                            
D₀⋅α₂⋅kₚ  + D₀⋅kₚ ⋅(α₂ - 1) - ⅈ⋅\omegaₚ⋅α₂ - ⅈ⋅\omegaₚ⋅(α₂ - 1) + α₂⋅m₁ + α₂⋅m

      2                                                                       
₂ + kₗ ⋅(D₀⋅α₂ + 2⋅D₀⋅(α₂ - 1)) + kₗ⋅(-2⋅D₀⋅α₂⋅kₚ - 2⋅D₀⋅kₚ⋅(α₂ - 1)) + 2⋅m₁⋅(

       
α₂ - 1)

In [48]:
Q = Poly(simpler, Symbol("k_l"))
Q

Poly((3*D_0*alpha_2 - 2*D_0)*k_l**2 + (-4*D_0*alpha_2*k_p + 2*D_0*k_p)*k_l + 2
*D_0*alpha_2*k_p**2 - D_0*k_p**2 - 2*I*\omega_p*alpha_2 + I*\omega_p + 3*alpha
_2*m_1 + alpha_2*m_2 - 2*m_1, k_l, domain='EX')

In [49]:
A, B = (Q.coeffs()[1]/Q.coeffs()[0]),(Q.coeffs()[2]/Q.coeffs()[0])
A,B

⎛                                 2        2                                  
⎜-4⋅D₀⋅α₂⋅kₚ + 2⋅D₀⋅kₚ  2⋅D₀⋅α₂⋅kₚ  - D₀⋅kₚ  - 2⋅ⅈ⋅\omegaₚ⋅α₂ + ⅈ⋅\omegaₚ + 3⋅
⎜─────────────────────, ──────────────────────────────────────────────────────
⎝    3⋅D₀⋅α₂ - 2⋅D₀                                   3⋅D₀⋅α₂ - 2⋅D₀          

                    ⎞
α₂⋅m₁ + α₂⋅m₂ - 2⋅m₁⎟
────────────────────⎟
                    ⎠

In [50]:
k = Symbol("k_l")
exps = k**2 +  A*k + B
exps

                                             2        2                       
  2   kₗ⋅(-4⋅D₀⋅α₂⋅kₚ + 2⋅D₀⋅kₚ)   2⋅D₀⋅α₂⋅kₚ  - D₀⋅kₚ  - 2⋅ⅈ⋅\omegaₚ⋅α₂ + ⅈ⋅\
kₗ  + ────────────────────────── + ───────────────────────────────────────────
            3⋅D₀⋅α₂ - 2⋅D₀                                       3⋅D₀⋅α₂ - 2⋅D

                               
omegaₚ + 3⋅α₂⋅m₁ + α₂⋅m₂ - 2⋅m₁
───────────────────────────────
₀                              

In [51]:
Poly(expand(exps.subs(k, k-A/2)), k )

Poly(k_l**2 - 4*D_0**2*alpha_2**2*k_p**2/(9*D_0**2*alpha_2**2 - 12*D_0**2*alph
a_2 + 4*D_0**2) + 4*D_0**2*alpha_2*k_p**2/(9*D_0**2*alpha_2**2 - 12*D_0**2*alp
ha_2 + 4*D_0**2) - D_0**2*k_p**2/(9*D_0**2*alpha_2**2 - 12*D_0**2*alpha_2 + 4*
D_0**2) + 2*D_0*alpha_2*k_p**2/(3*D_0*alpha_2 - 2*D_0) - D_0*k_p**2/(3*D_0*alp
ha_2 - 2*D_0) - 2*I*\omega_p*alpha_2/(3*D_0*alpha_2 - 2*D_0) + I*\omega_p/(3*D
_0*alpha_2 - 2*D_0) + 3*alpha_2*m_1/(3*D_0*alpha_2 - 2*D_0) + alpha_2*m_2/(3*D
_0*alpha_2 - 2*D_0) - 2*m_1/(3*D_0*alpha_2 - 2*D_0), k_l, domain='EX')

In [52]:
#I have divided accross and of course it elims but actually i should have squared it in the substituation?
simplify(simpler/Poly(simpler, Symbol("k_l")).all_coeffs()[0])

        2        2                                           2                
D₀⋅α₂⋅kₚ  + D₀⋅kₗ ⋅(3⋅α₂ - 2) - 2⋅D₀⋅kₗ⋅kₚ⋅(2⋅α₂ - 1) + D₀⋅kₚ ⋅(α₂ - 1) - ⅈ⋅\o
──────────────────────────────────────────────────────────────────────────────
                                                               D₀⋅(3⋅α₂ - 2)  

                                                             
megaₚ⋅α₂ - ⅈ⋅\omegaₚ⋅(α₂ - 1) + α₂⋅m₁ + α₂⋅m₂ + 2⋅m₁⋅(α₂ - 1)
─────────────────────────────────────────────────────────────
                                                             

In [53]:
#p2.reduced_parametric_integral

In [54]:
p2.used_chung_wu

False

In [55]:
p2._p

In [56]:
#assert we do not eliminate both params...